In [ ]:
# Process raw data about research income from HESA and create a table we can use for lookup in analyses

In [121]:
import os
import re
import pandas as pd
import numpy as np
import sys
import glob

%matplotlib inline

CURRENT_DIR = os.getcwd()
LIB_DIR = CURRENT_DIR + '/lib'
DATA_DIR = CURRENT_DIR + '/data'
RAW_DATA_DIR = DATA_DIR + '/raw'
PROCESSED_DATA_DIR = DATA_DIR + '/processed'

In [122]:
# Absolute or relative path to HESA file with data on research income
income_from_councils_file = LIB_DIR + "/HESA-UK-research-income-from-research-councils-2017_2018.csv"
income_from_other_sources_file = LIB_DIR + "/HESA-UK-research-income-other-sources-2017_2018.csv"

In [123]:
income_from_councils = pd.read_csv(income_from_councils_file, encoding = 'unicode_escape')
income_from_other_sources = pd.read_csv(income_from_other_sources_file, encoding = 'unicode_escape')

In [124]:
income_from_councils.set_index('UKPRN')
income_from_other_sources.set_index('UKPRN')
cols_to_keep = income_from_other_sources.columns.difference(income_from_councils.columns)
income = pd.concat([income_from_councils, income_from_other_sources[cols_to_keep]], axis=1, join='inner')

In [125]:
income.head(5)

,UKPRN,HE Provider,1a Biotechnology and Biological Sciences Research Council (BBSRC),1b Medical Research Council (MRC),1c Natural Environment Research Council (NERC),1d Engineering and Physical Sciences Research Council (EPSRC),1e Economic and Social Research Council (ESRC),1f Arts & Humanities Research Council (AHRC),1g Science & Technology Facilities Council (STFC),1h Other,...,14 Non-EU other,15 Total,2 UK-based charities (open competitive process),3 UK based charities (other),"4 UK central government bodies/local authorities, health and hospital authorities",5 UK central government tax credits for research and development expenditure,"6 UK industry, commerce and public corporations",7 UK other sources,8 EU government bodies,9 EU-based charities (open competitive process)
0,10007783,The University of Aberdeen,208,1154,65,6,77,0,0,1,...,36,16791,3620,610,8460,0,434,73,705,74
1,10007849,University of Abertay Dundee,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,10007856,Aberystwyth University,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,10000163,AECC University College,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,10000291,Anglia Ruskin University,0,0,0,0,0,0,0,0,...,14,315,19,0,113,0,100,0,19,0


In [126]:
# drop duplicate columns
#income = income.drop([income.columns[11] , income.columns[12]], axis="columns")

In [127]:
income.columns

Index(['UKPRN', 'HE Provider',
       '1a Biotechnology and Biological Sciences Research Council (BBSRC)',
       '1b Medical Research Council (MRC)',
       '1c Natural Environment Research Council (NERC)',
       '1d Engineering and Physical Sciences Research Council (EPSRC)',
       '1e Economic and Social Research Council (ESRC)',
       '1f Arts & Humanities Research Council (AHRC)',
       '1g Science & Technology Facilities Council (STFC)', '1h Other',
       '1i Total Research Councils',
       '10 EU industry, commerce and public corporations',
       '11 EU (excluding UK) other',
       '12 Non-EU-based charities (open competitive process)',
       '13 Non-EU industry, commerce and public corporations',
       '14 Non-EU other', '15 Total',
       '2 UK-based charities (open competitive process)',
       '3 UK based charities (other)',
       '4 UK central government bodies/local authorities, health and hospital authorities',
       '5 UK central government tax credits for re

In [128]:
income['total_£000s'] = income['1i Total Research Councils'] + income['15 Total']

In [129]:
income

,UKPRN,HE Provider,1a Biotechnology and Biological Sciences Research Council (BBSRC),1b Medical Research Council (MRC),1c Natural Environment Research Council (NERC),1d Engineering and Physical Sciences Research Council (EPSRC),1e Economic and Social Research Council (ESRC),1f Arts & Humanities Research Council (AHRC),1g Science & Technology Facilities Council (STFC),1h Other,...,15 Total,2 UK-based charities (open competitive process),3 UK based charities (other),"4 UK central government bodies/local authorities, health and hospital authorities",5 UK central government tax credits for research and development expenditure,"6 UK industry, commerce and public corporations",7 UK other sources,8 EU government bodies,9 EU-based charities (open competitive process),total_£000s
0,10007783,The University of Aberdeen,208,1154,65,6,77,0,0,1,...,16791,3620,610,8460,0,434,73,705,74,18302
1,10007849,University of Abertay Dundee,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,10007856,Aberystwyth University,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,10000163,AECC University College,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,10000291,Anglia Ruskin University,0,0,0,0,0,0,0,0,...,315,19,0,113,0,100,0,19,0,315
5,10007759,Aston University,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,10007857,Bangor University,0,0,0,0,0,0,0,0,...,343,233,0,54,0,0,44,12,0,343
7,10000571,Bath Spa University,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,10007850,The University of Bath,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,10007152,University of Bedfordshire,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [130]:
total_research_income = income['total_£000s'].sum()
income['total_%'] = (income['total_£000s']/total_research_income)*100

In [131]:
income

,UKPRN,HE Provider,1a Biotechnology and Biological Sciences Research Council (BBSRC),1b Medical Research Council (MRC),1c Natural Environment Research Council (NERC),1d Engineering and Physical Sciences Research Council (EPSRC),1e Economic and Social Research Council (ESRC),1f Arts & Humanities Research Council (AHRC),1g Science & Technology Facilities Council (STFC),1h Other,...,2 UK-based charities (open competitive process),3 UK based charities (other),"4 UK central government bodies/local authorities, health and hospital authorities",5 UK central government tax credits for research and development expenditure,"6 UK industry, commerce and public corporations",7 UK other sources,8 EU government bodies,9 EU-based charities (open competitive process),total_£000s,total_%
0,10007783,The University of Aberdeen,208,1154,65,6,77,0,0,1,...,3620,610,8460,0,434,73,705,74,18302,0.737224
1,10007849,University of Abertay Dundee,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
2,10007856,Aberystwyth University,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
3,10000163,AECC University College,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
4,10000291,Anglia Ruskin University,0,0,0,0,0,0,0,0,...,19,0,113,0,100,0,19,0,315,0.012689
5,10007759,Aston University,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
6,10007857,Bangor University,0,0,0,0,0,0,0,0,...,233,0,54,0,0,44,12,0,343,0.013816
7,10000571,Bath Spa University,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
8,10007850,The University of Bath,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
9,10007152,University of Bedfordshire,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000


In [132]:
income['total_%'].sum()

100.00000000000003

In [133]:
# Insert region for institution
hesa_UK_higher_education_providers = pd.read_csv("lib/HESA_UK_higher_education_providers.csv", encoding = "utf-8")
hesa_UK_higher_education_providers_region_mapping = dict(hesa_UK_higher_education_providers[['UKPRN', 'Region']].values) # create a dict for lookup
income['region'] = income['UKPRN'].map(hesa_UK_higher_education_providers_region_mapping,na_action="ignore")
income

,UKPRN,HE Provider,1a Biotechnology and Biological Sciences Research Council (BBSRC),1b Medical Research Council (MRC),1c Natural Environment Research Council (NERC),1d Engineering and Physical Sciences Research Council (EPSRC),1e Economic and Social Research Council (ESRC),1f Arts & Humanities Research Council (AHRC),1g Science & Technology Facilities Council (STFC),1h Other,...,3 UK based charities (other),"4 UK central government bodies/local authorities, health and hospital authorities",5 UK central government tax credits for research and development expenditure,"6 UK industry, commerce and public corporations",7 UK other sources,8 EU government bodies,9 EU-based charities (open competitive process),total_£000s,total_%,region
0,10007783,The University of Aberdeen,208,1154,65,6,77,0,0,1,...,610,8460,0,434,73,705,74,18302,0.737224,Scotland
1,10007849,University of Abertay Dundee,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.000000,Scotland
2,10007856,Aberystwyth University,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.000000,Wales
3,10000163,AECC University College,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.000000,South West
4,10000291,Anglia Ruskin University,0,0,0,0,0,0,0,0,...,0,113,0,100,0,19,0,315,0.012689,Eastern
5,10007759,Aston University,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.000000,West Midlands
6,10007857,Bangor University,0,0,0,0,0,0,0,0,...,0,54,0,0,44,12,0,343,0.013816,Wales
7,10000571,Bath Spa University,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.000000,South West
8,10007850,The University of Bath,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.000000,South West
9,10007152,University of Bedfordshire,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.000000,Eastern


In [134]:
# Save to file
processed_income_data_file = PROCESSED_DATA_DIR + "/processed_HESA-UK-research-income.csv"
income.to_csv(processed_income_data_file, encoding = "utf-8")
print("Saved processed HESA UK research income data to "+ processed_income_data_file)

Saved processed HESA UK research income data to /Users/alex/work/SSI/Code/carpentry-workshops-instructors-extractor/data/processed/processed_HESA-UK-research-income.csv
